# 국가문화유산포털 사진 크롤링

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import random
from tqdm import tqdm
import chromedriver_autoinstaller as ca

## 검색어 설정

In [29]:
# 검색어 입력
# 예) search_word = '서삼릉',  검색어 더블 클릭하고 입력하면 편리.
search_word = '서삼릉'

## 페이지 접속

In [30]:
# 페이지 접속
ca.install(True)
url = 'https://www.heritage.go.kr/main/'
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(5)
driver.maximize_window()
time.sleep(1)

In [31]:
# 검색
search_box = driver.find_element_by_xpath('//*[@id="searchWord2"]')
search_button = driver.find_element_by_xpath('//*[@id="btnSearchTop"]')

search_box.send_keys(search_word)
search_button.click()
time.sleep( random.uniform(1,3) )
driver.implicitly_wait(3)

<ipython-input-31-0e848bd8d230>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_box = driver.find_element_by_xpath('//*[@id="searchWord2"]')
<ipython-input-31-0e848bd8d230>:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_button = driver.find_element_by_xpath('//*[@id="btnSearchTop"]')


In [32]:
# 검색 결과에서 원하는 자료 링크 찾기
search_result = bs(driver.page_source, 'lxml')
search_result.title.text

'국가문화유산포탈 | 문화재 검색'

In [33]:
raw_search_result_list = search_result.find('ul', 'search-result type-list')
len(raw_search_result_list.find_all('li'))

4

In [34]:
search_result_list = raw_search_result_list.find_all('li')
for title in search_result_list:
    temp = title.find('h4').text
    print(temp)

고양 서삼릉 ( 高陽 西三陵 ) 
영천치일리인종태실 ( 永川治日里仁宗胎室 ) 
서울 효창공원 ( 서울 孝昌公園 ) 
구리 동구릉 ( 九里 東九陵 ) 


In [35]:
# 검색한 단어가 들어간 링크 골라내기
search_result_list = raw_search_result_list.find_all('li')
for i, title in enumerate(search_result_list):
    temp = title.find('h4').text
    if search_word in temp:
        link = search_result_list[i].find('a').attrs['href']
        print(link)

https://www.heritage.go.kr/heri/cul/culSelectDetail.do?pageNo=1_1_1_1&ccbaCpno=1333102000000


In [36]:
# 검색 결과 연관 페이지로 이동
driver.get(link)
driver.implicitly_wait(3)

## 사진 페이지 정보 담기

In [37]:
# 페이지 정보 담기
soup = bs(driver.page_source, 'lxml')
soup.title.text

'사적 고양 서삼릉 (高陽 西三陵) | 국가문화유산포탈 | 문화재 검색'

In [38]:
# 더보기 클릭
more_button = driver.find_element_by_xpath('//*[@id="img_list_area"]/a')
more_button.click()

<ipython-input-38-800de7d0738e>:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  more_button = driver.find_element_by_xpath('//*[@id="img_list_area"]/a')


In [39]:
img_list_area = soup.find('div', id='img_list_area')
len(img_list_area)

9

In [40]:
li = img_list_area.find_all('li')
len(li)

60

## 게시글 링크 수집

In [41]:
img_link_list = []
for num in li:
    temp = num.find('a').attrs['href'][-10:-3]
    link = f'https://www.heritage.go.kr/heri/cul/imgHeritage.do?ccimId={temp}&ccbaKdcd=13&ccbaAsno=02000000&ccbaCtcd=31'
    img_link_list.append(link)

len(img_link_list)

60

## 사진 다운

In [42]:
i = 0
for link in tqdm(img_link_list):
    driver.get(link)
    driver.implicitly_wait(3)

    # 다운로드 버튼 클릭
    down_button = driver.find_element_by_xpath('//*[@id="popup_wrap"]/div[2]/div/ul/li[2]/a')
    down_button.click()
    # 점점 시간 늘리면서 랜덤(1~3초 사이)하게 쉬기(크롤링 차단 방지 위해 설정)
    time.sleep( random.uniform(1,3) + i)
    i += 0.01
# chromedriver 종료
driver.quit()

  0%|          | 0/60 [00:00<?, ?it/s]<ipython-input-42-5f7548ed76d1>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  down_button = driver.find_element_by_xpath('//*[@id="popup_wrap"]/div[2]/div/ul/li[2]/a')
100%|██████████| 60/60 [03:10<00:00,  3.18s/it]


# 코드 마지막